
# MCP: acceso a **servidores existentes** (TIME + FETCH [+ GIT opcional])

Esta notebook muestra **cómo conectarse desde Python a servidores MCP de uso general** usando el *cliente STDIO* del SDK oficial (`mcp`).  
Incluye **dos demos listos** y un bloque **opcional**:

- ⏰ **Time** (Python): hora actual y conversión de zonas horarias.
- 🌐 **Fetch** (Node/TS): traer contenido web como Markdown (si tenés `node`/`npx`).
- 🐙 **Git** (Python, opcional): lista herramientas y explica cómo invocarlas.

> Pensado para clase: cada sección imprime el **esquema de tools** para que estudiantes vean cómo inspeccionar y llamar herramientas expuestas por un servidor MCP.


In [2]:

# --- Instalación (ejecutá si faltan dependencias) ---
# Recomendado: un entorno limpio (uv/pipx/venv). Descomentar si hace falta instalar.
# !pip install -U "mcp[cli]" mcp-server-time mcp-server-git nest_asyncio

import sys, importlib, subprocess, json, os, shutil

def _which(cmd):
    return shutil.which(cmd) is not None

print("✔ Python:", sys.version.split()[0])
print("✔ node:", subprocess.getoutput("node -v") if _which("node") else "no encontrado")
print("✔ npx:", subprocess.getoutput("npx -v") if _which("npx") else "no encontrado")

def _import_ok(pkg):
    try:
        importlib.import_module(pkg)
        return True
    except Exception as e:
        print(f"⚠ Falta instalar: {pkg} ({e.__class__.__name__}: {e})")
        return False

have_mcp = _import_ok("mcp")
have_time = _import_ok("mcp_server_time")
# git es opcional, puede no estar instalado
try:
    importlib.import_module("mcp_server_git")
    have_git = True
except Exception:
    have_git = False

print({"mcp": have_mcp, "mcp_server_time": have_time, "mcp_server_git": have_git})


✔ Python: 3.10.15
✔ node: v20.17.0
✔ npx: 10.8.3
{'mcp': True, 'mcp_server_time': True, 'mcp_server_git': True}


In [3]:

# --- Utilidades de cliente STDIO ---
import asyncio
import nest_asyncio
nest_asyncio.apply()

from typing import Optional, Dict, Any, List

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def connect_stdio(command: str, args: List[str], env: Optional[Dict[str, str]] = None):
    '''
    Conecta a un servidor MCP vía STDIO lanzándolo como subproceso.
    Devuelve (session, aexit) donde `aexit()` cierra la conexión.
    '''
    params = StdioServerParameters(command=command, args=args, env=env or {})
    cm = stdio_client(params)
    read, write = await cm.__aenter__()
    session_cm = ClientSession(read, write)
    session = await session_cm.__aenter__()
    await session.initialize()
    # Describe capabilities
    tools = await session.list_tools()
    resources = await session.list_resources()
    print(f"🧩 Tools disponibles: {[t.name for t in tools.tools]}")
    print(f"📚 Resources: {[r.uri for r in resources.resources]}")
    async def aexit():
        await session_cm.__aexit__(None, None, None)
        await cm.__aexit__(None, None, None)
    return session, aexit

async def call_tool(session: ClientSession, name: str, args: Dict[str, Any]):
    '''Invoca una tool por nombre con manejo de errores legibles.'''
    print(f"\n▶ Llamando tool `{name}` con args={args}")
    try:
        result = await session.call_tool(name, args)
        print("✅ Resultado:")
        for item in (result.content or []):
            # item puede ser TextContent, Image, Blob, etc.
            if hasattr(item, "text") and item.text is not None:
                print(item.text)
            else:
                print(item)
        return result
    except Exception as e:
        print(f"❌ Error al invocar `{name}`: {e.__class__.__name__}: {e}")
        raise

def pick_arg_name(schema_props: Dict[str, Any], candidates: List[str], default: Optional[str] = None):
    '''Devuelve el primer nombre de argumento presente en el schema.properties.'''
    keys = set((schema_props or {}).keys())
    for c in candidates:
        if c in keys:
            return c
    return default



## ⏰ Demo 1 — Time (Python)

Usamos el **Time server** para obtener la hora actual y convertir horarios entre zonas.  
Este servidor existe como paquete PyPI y se puede ejecutar como módulo con `python -m mcp_server_time`.


In [ ]:

async def demo_time():
    session, done = await connect_stdio(
        command="python",
        args=["-m", "mcp_server_time"]
    )
    try:
        # Intento 1: get_current_time(timezone=...)
        timezone = "America/Argentina/Buenos_Aires"
        await call_tool(session, "get_current_time", {"timezone": timezone})

        # Intento 2: convert_time / convert_timezone con nombres alternativos de args
        # Inspeccionamos el schema de la tool para adaptar nombres
        tools = await session.list_tools()
        convert_tool = None
        for t in tools.tools:
            if t.name in ("convert_time", "convert_timezone"):
                convert_tool = t
                break

        if convert_tool:
            props = (convert_tool.inputSchema or {}).get("properties", {})
            # Posibles variaciones de nombres
            src_key = pick_arg_name(props, ["source_timezone", "from_timezone", "from_tz"], "source_timezone")
            tgt_key = pick_arg_name(props, ["target_timezone", "to_timezone", "to_tz"], "target_timezone")
            time_key = pick_arg_name(props, ["time", "datetime", "at"], "time")
            fmt_key  = pick_arg_name(props, ["format", "fmt"], None)

            args = {
                time_key: "2025-08-19T14:00:00",
                src_key: "America/Argentina/Buenos_Aires",
                tgt_key: "Europe/Madrid",
            }
            if fmt_key:
                args[fmt_key] = "%Y-%m-%d %H:%M:%S"
            await call_tool(session, convert_tool.name, args)
        else:
            print("ℹ No encontré tool de conversión; sólo se mostró la hora actual.")
    finally:
        await done()

# Ejecutar la demo
import asyncio
asyncio.get_event_loop().run_until_complete(demo_time())



## 🌐 Demo 2 — Fetch (Node/TypeScript, opcional)

Si tenés **Node + npx**, podés usar el **Fetch server** para traer páginas web y convertirlas a Markdown.

> Paquete sugerido: `@modelcontextprotocol/server-fetch` (se ejecuta con `npx -y @modelcontextprotocol/server-fetch`).  
> Si no tenés Node, salteá esta celda.


In [ ]:

import shutil

def have_npx():
    return shutil.which("npx") is not None

async def demo_fetch():
    if not have_npx():
        print("⚠ No hay `npx` en el PATH. Instalá Node.js para correr esta demo.")
        return

    # Lanza el server TypeScript por STDIO usando npx
    session, done = await connect_stdio(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-fetch"]
    )
    try:
        # Tool típica: `fetch` con argumento `url`
        await call_tool(session, "fetch", {"url": "https://example.com"})
    finally:
        await done()

asyncio.get_event_loop().run_until_complete(demo_fetch())



## 🐙 Demo 3 — Git (Python, opcional)

El **Git server** ofrece herramientas para inspeccionar y operar repositorios Git.
Para usarlo cómodamente, **ejecutá esta notebook dentro de un repo** o indicá ruta del repo si la tool lo pide.

> Paquete PyPI: `mcp-server-git` → ejecución: `python -m mcp_server_git`  
> Las tools disponibles pueden cambiar entre versiones; por eso **listamos el esquema** y te dejamos un ejemplo editable.


In [ ]:

REPO_PATH = "."  # Cambiá a un repo válido si lo necesitás, p.ej. "../python-sdk"

async def demo_git():
    try:
        session, done = await connect_stdio(
            command="python",
            args=["-m", "mcp_server_git"],
            env={"REPO_PATH": REPO_PATH}  # algunas implementaciones lo utilizan
        )
    except Exception as e:
        print("⚠ No se pudo iniciar mcp_server_git (¿instalado? ¿repo válido?).", e)
        return

    try:
        # Mostramos tools y schemas
        tools = await session.list_tools()
        for t in tools.tools:
            print(f"\n— {t.name} —")
            print("Descripción:", getattr(t, "description", ""))
            print("Schema:", (t.inputSchema or {}))

        # 🔧 EJEMPLO editable: elegí una tool del listado e indicá argumentos
        # (algunos servidores esperan algo como {"path": ".", "pattern": "*.py"})
        # result = await call_tool(session, "list_files", {"path": ".", "pattern": "*.py"})
        # print(result)

    finally:
        await done()

asyncio.get_event_loop().run_until_complete(demo_git())



### Consejos didácticos

- **Exploración guiada**: Pedí a los alumnos que comparen el *schema* de una tool con los argumentos que arma el cliente.  
- **Ejercicio 1 (5’)**: en *Time*, cambiar zonas horarias y formato de salida.  
- **Ejercicio 2 (10’)**: en *Fetch*, traer una noticia y limitar el contenido (si el server soporta paginado/índices).  
- **Ejercicio 3 (10’)**: en *Git*, listar archivos `*.ipynb` del repo actual (o buscar en el historial si la tool existe).

> Seguridad: Al usar servers de terceros, revisá su procedencia y permisos. Evitá exponer *tokens* por env vars en máquinas de alumnos.
